# December 25, 2023

https://adventofcode.com/2023/day/25

In [4]:
import numpy as np
from collections import defaultdict

In [5]:
def parse_input(text):
    graph = defaultdict(list)
    
    for line in text:
        key, edges = line.split(": ")
        for e in edges.split(" "):
            graph[key].append(e)
            graph[e].append(key)

    return graph

In [6]:
text = f'''jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr'''

test = parse_input(text.split("\n"))

In [7]:
fn = "../data/2023/25.txt"
with open(fn, "r") as file:
    text = file.readlines()
text = [line.strip() for line in text]
puzz = parse_input(text)

In [107]:
class Graph:
    def __init__(self, d, n=None):
        self.edges = d # dict
        if n is None:
            self.n = sum( [len(v) for v in self.edges.values()] )
        else:
            self.n = n

    def random_cut( self ):
        idx = np.random.randint( self.n )
        for s, v in self.edges.items():
            if idx >= len(v):
                idx -= len(v)
            else:
                t = v[idx]
                break
    
        #print("collapse", s, t)
        # collapse edge s,t
        new_dict = self.edges.copy()
        new_dict[s] = [ e for e in new_dict[s] if e != t ]
        new_dict[t] = [ e for e in new_dict[t] if e != s ]
        new_dict[s+t] = new_dict[s] + new_dict[t]
        del new_dict[s]
        del new_dict[t]

        # move edges to new node
        for k,v in new_dict.items():
            new_dict[k] = [e if e not in [s,t] else s+t for e in v]
        return Graph( new_dict )
    
    def Karger( self ):
        g2 = Graph( self.edges )
        while len(g2.edges) > 2:
            g2 = g2.random_cut()
        return g2

    def part1( self ):
        i = 0
        while True:
            i += 1
            print("Attempt", i)

            ans = self.Karger()
            if sum( [len(v) for v in ans.edges.values()] ) <= 3:
                return ans

        


In [247]:
class Graph():
    def __init__(self, edges):

        # get list of all nodes
        self.nodes = set()
        for nodes in edges.items():
            self.nodes |= set( [nodes[0]] + nodes[1] ) # include both key and each value in the node set
        self.nodes = list(self.nodes)
        self.N = len(self.nodes)

        # create mapping of node to array index
        self.index = { node:i for i,node in enumerate(self.nodes)}

        # create adj matrix

        self.adj = np.identity(self.N, dtype=int)
        for s, nbrs in edges.items():
            for t in nbrs:
                self.adj[ self.index[s], self.index[t] ] = 1
                self.adj[ self.index[t], self.index[s] ] = 1


In [314]:
def MergeNodes( adj_mat, s, t ):
    # ensure s < t for simplicity
    if t < s:
        tmp = s
        s = t
        t = tmp

    #print(adj_mat.shape)
    # initialize new_mat sans removed node, t
    new_mat = np.delete(adj_mat, t, axis=0) # deep copy
    new_mat = np.delete(new_mat, t, axis=1)
    #print(adj_mat.shape)
    # count total number of links to s or t
    st_col = adj_mat[s] + adj_mat[t]
    st_col = np.delete(st_col, t)
    # for the diagonal, we'll keep track of how many nodes are in these conglomerates
    st_col[s] = adj_mat[s,s] + adj_mat[t,t]
    new_mat[s, :] = st_col
    new_mat[:, s] = st_col

    return new_mat

def MinimumCutPhase( adj_mat, a):
    A = [a]
    while (len(A) < adj_mat.shape[0]):

        # Find connectedness to all nodes in A
        weights = np.sum( adj_mat[A], axis=0 )

        # Find most connected node outside of A
        best = 0
        idx = None
        for i, w in enumerate(weights):
            if i in A:
                continue
            if w > best:
                best = w
                idx = i

        # Add that node to A
        A.append(idx)

    # Find the s-t
    #print(A)
    s = A[-2]
    t = A[-1]

    # cut for this phase is all but the last node
    cut_of_the_phase = A[:-1]
    # The flow of this cut is the connectedness of that last node to A
    phase_flow = best
    # new graph with those merged
    new_mat = MergeNodes(adj_mat, s, t)

    return new_mat, cut_of_the_phase, phase_flow

def MinimumCut( adj_mat, a=0 ):
    # initialize best to be worse than
    # the worst case scenario, size of graph - 1
    best_flow = adj_mat.shape[0]
    best_cut = []
    best_graph = None

    # keep track of total nodes for final calculation
    total_nodes = adj_mat.shape[0]

    i = 0
    while adj_mat.shape[0] > 1:
        i+=1
        if i % 1000 >= 0:
            print(i, end="")
        new_mat, new_cut, new_flow = MinimumCutPhase(adj_mat, a)
        if new_flow < best_flow:
            best_flow = new_flow
            best_cut = new_cut
            best_graph = adj_mat # if this is best flow, we want the pre-merged graph
            print(" best flow:", best_flow)
        else:
            print("")
        adj_mat = new_mat
        if best_flow == 3:
            break

    # We now have the best cut, let's figure out the number of nodes on each side
    one_side = sum( [best_graph[i,i] for i in best_cut] )
    two_side = total_nodes - one_side
    
    return one_side*two_side, best_graph, best_cut, best_flow



        

In [ ]:
for idx

In [ ]:
def find_triangles( graph ):
    nodes = list(graph.keys())
    index = { n:i for i,n in enumerate(nodes) }

    triangles = defaultdict(list)

    for idx in range(len(index)):
        edge = nodes[idx]
        nbrs = graph[edge]

        for ni in range(len(nbrs)):
            s = nbrs[ni]
            sdx = index[s]
            # we only look for triangles involving higher numbered nodes
            # since we already did lower numbered nodes in a previous loop
            if sdx < idx:
                continue

            for nj in range(ni+1, len(nbrs)):
                t = nbrs[nj]
                tdx = index[t]
                if tdx < idx:
                    continue

                if t in graph[s]:
                    triangles[edge].append( [s,t] )
                    triangles[s].append( [edge,t] )
                    triangles[t].append( [s,edge] )



In [315]:
Gtest = Graph(test)

In [316]:
out_test = MinimumCut(Gtest.adj)
out_test

1 best flow: 4
2
3
4
5
6
7 best flow: 3


(54, array([[1, 1, 0, 0, 0, 1, 0, 1, 1],
        [1, 1, 0, 1, 0, 0, 1, 0, 1],
        [0, 0, 6, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0],
        [1, 0, 1, 0, 1, 1, 0, 1, 1],
        [0, 1, 0, 1, 1, 0, 1, 2, 0],
        [1, 0, 0, 1, 0, 1, 2, 2, 1],
        [1, 1, 1, 0, 0, 1, 0, 1, 1]]), [0, 1, 8, 5, 7, 6, 3, 4], 3)

In [317]:
Gpuzz = Graph(puzz)
out_puzz = MinimumCut(Gpuzz.adj)
out_puzz

1 best flow: 4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

(612945, array([[  1,   0,   0, ...,   0,   0,   0],
        [  0, 771,   0, ...,   0,   0,   0],
        [  0,   0,   2, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   2,   0,   0],
        [  0,   0,   0, ...,   0,   1,   0],
        [  0,   0,   0, ...,   0,   0,   1]]), [0,
  5,
  8,
  32,
  52,
  70,
  79,
  91,
  120,
  162,
  122,
  160,
  14,
  82,
  25,
  35,
  93,
  51,
  60,
  251,
  385,
  403,
  488,
  250,
  48,
  71,
  73,
  77,
  97,
  23,
  43,
  27,
  74,
  362,
  187,
  24,
  478,
  536,
  33,
  426,
  9,
  375,
  38,
  514,
  47,
  471,
  176,
  53,
  37,
  166,
  19,
  341,
  130,
  235,
  382,
  398,
  452,
  371,
  474,
  484,
  139,
  96,
  115,
  397,
  199,
  266,
  193,
  188,
  327,
  361,
  226,
  416,
  431,
  450,
  451,
  466,
  357,
  244,
  369,
  312,
  378,
  127,
  321,
  161,
  178,
  272,
  387,
  330,
  392,
  493,
  118,
  26,
  135,
  202,
  394,
  180,
  192,
  373,
  414,
  116,
  252,
  133,
  246,
  423,
  183,
  173,
  19